# ScanImage Metadata Comparison

Test datasets: `\\2025-12-18_metadata-test`

This notebook compares metadata differences between ScanImage acquisition variants:
1. Uniform sampling ON vs OFF
2. Frame averaging (1 vs 10 vs 100)
3. Galvo type (RGG vs GG)
4. Single vs dual color channel (LBM)
5. LBM vs Piezo stack
6. Piezo stack: single vs dual channel

In [ ]:
# !uv pip install pandas
from pathlib import Path
import pandas as pd
import mbo_utilities as mbo

base_path = Path(r"2025-12-18_metadata-test")

# TODO: We should define why these are needed
# e.g. XResolution is an ImageJ requirement
EXCLUDE_ALIASES = (
    # dx/dy/dz aliases
    "PhysicalSizeX", "PhysicalSizeY", "PhysicalSizeZ",
    "pixel_resolution", "voxel_size", "z_step",
    # fs aliases
    "frame_rate", "fr", "sampling_frequency", "frameRate", "scanFrameRate", "fps", "vps",
    # num_timepoints aliases
    "nframes", "num_frames", "n_frames",
    # num_zplanes aliases
    "num_planes", "nplanes", "Z", "nz", "zplanes",
    # shape/size aliases
    "array_shape", "data_shape", "shape",
    # roi aliases
    "roi_px", "roi_size", "fov_pixels", "fov_px",
    # file paths (always different)
    "file_paths",
)


def flatten_dict(d, parent_key="", sep="."):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep).items())
        else:
            items.append((new_key, v))
    return dict(items)


def metadata_diff(meta_a, meta_b, labels=("A", "B"), exclude=()):
    flat_a = flatten_dict(meta_a)
    flat_b = flatten_dict(meta_b)

    all_keys = set(flat_a.keys()) | set(flat_b.keys())
    differences = []

    for key in sorted(all_keys):
        # skip excluded keys (partial match)
        if any(exc in key for exc in exclude):
            continue

        val_a = flat_a.get(key)
        val_b = flat_b.get(key)
        if val_a != val_b:
            differences.append({
                "parameter": key,
                labels[0]: val_a,
                labels[1]: val_b,
            })

    return pd.DataFrame(differences)


def print_stack_type(arr, name):
    """print lbm_stack and piezo_stack values."""
    meta = arr.metadata
    print(f"{name}: lbm_stack={meta.get('lbm_stack')}, piezo_stack={meta.get('piezo_stack')}")

## Test Files

| Directory | Configuration | Key Features |
|-----------|---------------|--------------|
| `lbm/` | LBM single channel | uniform sampling ON/OFF, frame avg 1/10/100, RGG/GG |
| `dual-channel-lbm/` | LBM dual channel | AI0 + AI1 |
| `stack_averaging/` | Piezo stack | `logAverageFactor>1` |
| `dual-channel-stack/` | Piezo + 2 colors | Standard piezo with 2 channels |

## 1. Uniform Sampling: ON vs OFF

X-Galvo samples at uniform velocity, reducing edge distortions on mROI edges.

In [16]:
uniform_on = base_path / "lbm" / "lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling-ON_Avg1_00001.tif"
uniform_off = base_path / "lbm" / "lbm_AI0-14ch_2mROIs_184px_RGG_uniform-sampling-OFF_avg1_00001.tif"

arr_on = mbo.imread(uniform_on)
arr_off = mbo.imread(uniform_off)

print(f"Uniform ON:  {uniform_on.name}  shape={arr_on.shape}")
print(f"Uniform OFF: {uniform_off.name}  shape={arr_off.shape}")
print_stack_type(arr_on, "uniform_ON")
print_stack_type(arr_off, "uniform_OFF")
print()

metadata_diff(
    arr_on.metadata,
    arr_off.metadata,
    labels=("uniform_ON", "uniform_OFF"),
    exclude=EXCLUDE_ALIASES,
)

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

Uniform ON:  lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling-ON_Avg1_00001.tif  shape=(100, 14, 448, 448)
Uniform OFF: lbm_AI0-14ch_2mROIs_184px_RGG_uniform-sampling-OFF_avg1_00001.tif  shape=(100, 14, 368, 368)
uniform_ON: lbm_stack=True, piezo_stack=False
uniform_OFF: lbm_stack=True, piezo_stack=False



,parameter,uniform_ON,uniform_OFF
0,dx,2.0,2.43
1,dy,2.0,2.43
2,fov,"(448, 448)","(368, 368)"
3,fs,17.07,20.63
4,num_elements,204288,138368
5,page_height,912,752
6,page_width,224,184
7,roi,"(224, 448)","(184, 368)"
8,roi_groups,"[{'ver': 1, 'classname': 'scanimage.mroi.Roi',...","[{'ver': 1, 'classname': 'scanimage.mroi.Roi',..."
9,roi_heights,"[448, 448]","[368, 368]"


## 2. Frame Averaging: 1 vs 10 vs 100

`logAverageFactor` controls how many frames are averaged before saving.

In [17]:
avg1 = base_path / "lbm" / "lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling-ON_Avg1_00001.tif"
avg10 = base_path / "lbm" / "lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling-ON_Avg10_00001.tif"
avg100 = base_path / "lbm" / "lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling-ON_Avg100_00001.tif"

arr_avg1 = mbo.imread(avg1)
arr_avg10 = mbo.imread(avg10)

print(f"Avg1:  {avg1.name}  shape={arr_avg1.shape}")
print(f"Avg10: {avg10.name}  shape={arr_avg10.shape}")
print_stack_type(arr_avg1, "avg1")
print_stack_type(arr_avg10, "avg10")
print()

metadata_diff(
    arr_avg1.metadata,
    arr_avg10.metadata,
    labels=("avg1", "avg10"),
    exclude=EXCLUDE_ALIASES,
)

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

Avg1:  lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling-ON_Avg1_00001.tif  shape=(100, 14, 448, 448)
Avg10: lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling-ON_Avg10_00001.tif  shape=(10, 14, 448, 448)
avg1: lbm_stack=True, piezo_stack=False
avg10: lbm_stack=True, piezo_stack=False



,parameter,avg1,avg10
0,num_timepoints,100,10
1,si.hChannels.channelOffset,"[-397, -113]","[-401, -113]"
2,si.hRoiManager.linePeriod,0.000063,0.000063
3,si.hRoiManager.scanFramePeriod,0.058571,0.05857
4,si.hRoiManager.scanVolumeRate,17.073427,17.073651
5,si.hScan2D.channelOffsets,"[-397, -113]","[-401, -113]"
6,si.hScan2D.logAverageFactor,1,10
7,si.hScan2D.scannerFrequency,7922.069913,7922.174135


In [18]:
arr_avg100 = mbo.imread(avg100)

print(f"Avg1:   {avg1.name}  shape={arr_avg1.shape}")
print(f"Avg100: {avg100.name}  shape={arr_avg100.shape}")
print_stack_type(arr_avg1, "avg1")
print_stack_type(arr_avg100, "avg100")
print()

metadata_diff(
    arr_avg1.metadata,
    arr_avg100.metadata,
    labels=("avg1", "avg100"),
    exclude=EXCLUDE_ALIASES,
)

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

Avg1:   lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling-ON_Avg1_00001.tif  shape=(100, 14, 448, 448)
Avg100: lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling-ON_Avg100_00001.tif  shape=(1, 14, 448, 448)
avg1: lbm_stack=True, piezo_stack=False
avg100: lbm_stack=True, piezo_stack=False



,parameter,avg1,avg100
0,num_timepoints,100,1
1,si.hChannels.channelOffset,"[-397, -113]","[-404, -109]"
2,si.hRoiManager.linePeriod,0.000063,0.000063
3,si.hRoiManager.scanFramePeriod,0.058571,0.05857
4,si.hRoiManager.scanVolumeRate,17.073427,17.073555
5,si.hScan2D.channelOffsets,"[-397, -113]","[-404, -109]"
6,si.hScan2D.logAverageFactor,1,100
7,si.hScan2D.scannerFrequency,7922.069913,7922.129468


## 3. Galvo Type: RGG (Resonant) vs GG (Galvo-Galvo)

Without the fast galvo, the fps and image dimensions are substantially reduced.

In [19]:
rgg_file = base_path / "lbm" / "lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling-ON_Avg1_00001.tif"
gg_file = base_path / "lbm" / "lbm_AI0-14ch_2mROIs_512px_GG_avg1_00001.tif"

arr_rgg = mbo.imread(rgg_file)
arr_gg = mbo.imread(gg_file)

print(f"RGG: {rgg_file.name}  shape={arr_rgg.shape}")
print(f"GG:  {gg_file.name}  shape={arr_gg.shape}")
print_stack_type(arr_rgg, "RGG")
print_stack_type(arr_gg, "GG")
print()

metadata_diff(
    arr_rgg.metadata,
    arr_gg.metadata,
    labels=("RGG", "GG"),
    exclude=EXCLUDE_ALIASES,
)

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

RGG: lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling-ON_Avg1_00001.tif  shape=(100, 14, 448, 448)
GG:  lbm_AI0-14ch_2mROIs_512px_GG_avg1_00001.tif  shape=(20, 14, 1024, 1024)
RGG: lbm_stack=True, piezo_stack=False
GG: lbm_stack=True, piezo_stack=False



,parameter,RGG,GG
0,dx,2.0,0.88
1,dy,2.0,0.88
2,fov,"(448, 448)","(1024, 1024)"
3,fs,17.07,0.52
4,num_elements,204288,1048576
5,num_fly_to_lines,16,1
6,num_timepoints,100,20
7,page_height,912,2048
8,page_width,224,512
9,roi,"(224, 448)","(512, 1024)"


## 4. LBM Color Channels: Single vs Dual

Color channels detected by counting unique AI sources in `virtualChannelSettings`.

In [20]:
single_ch = base_path / "lbm" / "lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling-ON_Avg1_00001.tif"
dual_ch = base_path / "dual-channel-lbm" / "lbm_AI0-14ch_AI1-3ch_2mROIs_224px_RGG_uniform-sampling-ON_00001.tif"

arr_single = mbo.imread(single_ch)
arr_dual = mbo.imread(dual_ch)

print(f"Single channel: {single_ch.name}  shape={arr_single.shape}")
print(f"Dual channel:   {dual_ch.name}  shape={arr_dual.shape}")
print_stack_type(arr_single, "single_ch")
print_stack_type(arr_dual, "dual_ch")
print()

metadata_diff(
    arr_single.metadata,
    arr_dual.metadata,
    labels=("single_ch", "dual_ch"),
    exclude=EXCLUDE_ALIASES,
)

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

Single channel: lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling-ON_Avg1_00001.tif  shape=(100, 14, 448, 448)
Dual channel:   lbm_AI0-14ch_AI1-3ch_2mROIs_224px_RGG_uniform-sampling-ON_00001.tif  shape=(10, 17, 448, 448)
single_ch: lbm_stack=True, piezo_stack=False
dual_ch: lbm_stack=True, piezo_stack=False



,parameter,single_ch,dual_ch
0,color_channels,1,2
1,n_planes,14,17
2,ncolors,1,2
3,numPlanes,14,17
4,num_color_channels,1,2
5,num_colors,1,2
6,num_timepoints,100,10
7,num_z,14,17
8,planes,14,17
9,si.hChannels.channelOffset,"[-397, -113]","[-339, -127]"


## 5. Stack Type: LBM vs Piezo

LBM uses `channelSave` for z-planes, Piezo uses `hStackManager`.

In [21]:
lbm_file = base_path / "lbm" / "lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling-ON_Avg1_00001.tif"
piezo_file = base_path / "stack_averaging" / "stack_AI0-1ch_3xZoom_224px_RGG_uniform-sampling-ON_17volume_11slices_10frames-per-slice_Avg10_00001.tif"

arr_lbm = mbo.imread(lbm_file)
arr_piezo = mbo.imread(piezo_file)

print(f"LBM:   {lbm_file.name}  shape={arr_lbm.shape}")
print(f"Piezo: {piezo_file.name}  shape={arr_piezo.shape}")
print_stack_type(arr_lbm, "LBM")
print_stack_type(arr_piezo, "Piezo")
print()

metadata_diff(
    arr_lbm.metadata,
    arr_piezo.metadata,
    labels=("LBM", "Piezo"),
    exclude=EXCLUDE_ALIASES,
)

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

LBM:   lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling-ON_Avg1_00001.tif  shape=(100, 14, 448, 448)
Piezo: stack_AI0-1ch_3xZoom_224px_RGG_uniform-sampling-ON_17volume_11slices_10frames-per-slice_Avg10_00001.tif  shape=(187, 1, 224, 224)
LBM: lbm_stack=True, piezo_stack=False
Piezo: lbm_stack=False, piezo_stack=True



,parameter,LBM,Piezo
0,dx,2.0,2.13
1,dy,2.0,2.13
2,dz,None,7.0
3,fov,"(448, 448)","(224, 224)"
4,fov_micrometers,"(448, 896)","(477, 477)"
5,fov_um,"(448, 896)","(477, 477)"
6,fs,17.07,66.02
7,is_lbm,True,False
8,is_piezo,False,True
9,lbmStack,True,False


## 7. Summary: get_stack_info() Output

Quick summary of detected stack parameters across all test files.

In [ ]:
from mbo_utilities.metadata.scanimage import get_stack_info

# collect all test files
all_files = []
for subdir in ["lbm", "dual-channel-lbm", "stack_averaging", "dual-channel-stack"]:
    path = base_path / subdir
    if path.exists():
        for f in sorted(path.glob("*.tif")):
            all_files.append((subdir, f))

# build summary table
rows = []
for subdir, f in all_files:
    arr = mbo.imread(f)
    info = get_stack_info(arr.metadata)
    rows.append({
        "dir": subdir,
        "file": f.name[:50] + "..." if len(f.name) > 50 else f.name,
        "shape": str(arr.shape),
        "stack_type": info["stack_type"],
        "num_zplanes": info["num_zplanes"],
        "num_colors": info["num_color_channels"],
        "fps": info["frames_per_slice"],
        "logAvg": info["log_average_factor"],
        "dz": info["dz"],
        "fs": info["fs"],
    })

pd.DataFrame(rows)

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

...

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

,dir,file,shape,stack_type,num_zplanes,num_colors,fps,logAvg,dz,fs
0,lbm,lbm_AI0-14ch_2mROIs_184px_RGG_uniform-sampling...,"(1, 14, 368, 368)",lbm,14,1,100,100,5.0,20.63
1,lbm,lbm_AI0-14ch_2mROIs_184px_RGG_uniform-sampling...,"(100, 14, 368, 368)",lbm,14,1,100,1,5.0,20.63
2,lbm,lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling...,"(1, 14, 448, 448)",lbm,14,1,100,100,5.0,17.07
3,lbm,lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling...,"(10, 14, 448, 448)",lbm,14,1,100,10,5.0,17.07
4,lbm,lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling...,"(100, 14, 448, 448)",lbm,14,1,100,1,5.0,17.07
5,lbm,lbm_AI0-14ch_2mROIs_512px_GG_avg1_00001.tif,"(20, 14, 1024, 1024)",lbm,14,1,20,1,5.0,0.52
6,lbm,lbm_AI0-14ch_2mROIs_512px_GG_avg5_00001.tif,"(4, 14, 1024, 1024)",lbm,14,1,20,5,5.0,0.52
7,dual-channel-lbm,lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling...,"(10, 14, 448, 448)",lbm,14,2,10,1,7.0,17.07
8,dual-channel-lbm,lbm_AI0-14ch_AI1-3ch_2mROIs_224px_RGG_uniform-...,"(10, 17, 448, 448)",lbm,17,2,10,1,7.0,17.07
9,stack_averaging,stack_AI0-1ch_3xZoom_224px_RGG_uniform-samplin...,"(187, 1, 224, 224)",piezo,11,1,10,10,7.0,66.02
